# Kalman Filtering

In [1]:
import Simulator
import Observation
import Statistics
import KalmanFilter

## Loading simulator 

For Kalman filtering we have to be able to propagate a Gaussian distribution in time and to have access to the linear operators

In [2]:
timestamp = "2021_08_10-09_34_31"

grid, simulator = Simulator.from_file(timestamp)

## Definition of initial distribution 

The inital state is equipped with a Gaussian distribution specified by its mean and covariance. 

In [3]:
statistics = Statistics.Statistics(simulator)

Please remember to set priors!


In [4]:
prior_args = Statistics.prior_args_from_file(timestamp)
statistics.set_prior(prior_args)

statistics.plot()

## Loading observations 

Observation positions which are fixed for a setup and values from the truth

In [ ]:
obs_timestamp = "2021_08_10-09_34_39"
observation = Observation.from_file(grid, timestamp, obs_timestamp)

## Kalman filtering

Updating/conditioning the distributions at every observation time

In [ ]:
kalmanFilter = KalmanFilter.Kalman(statistics, observation)

for t in range(observation.N_obs):
    print(t)
    statistics.propagate(25)
    kalmanFilter.filter(statistics.mean, statistics.cov, observation.obses[t])
    statistics.plot()